## Einleitung

Automatisierte Objekterkennung im Allgemeinen und Gesichtserkenung und -analyse im Speziellen funktioniert mittlerweile erstaunlich gut. Für diese Aufgaben werden heute praktisch ausschliesslich neuronale Netze (meistens CNNs oder RNNs) verwendet.

Die verfügbaren Cloud-Dienste sind zum Teil so mächtig, dass man sie für viele Probleme out-of-the box verwenden kann. In diesem Toy Project werden wir das Google Cloud Vision API verwenden. Die Daten dazu holen wir uns von flickr.

Aber auch Libraries, die man offline verwenden kann, sind schon erstaunlich gut. Wir verwenden die Python-Bibliothek face_recognition, um die Daten von flickr ein wenig zu säubern.

## Fragestellung und Metrik
In diesem fiktiven Projekt wurde uns aufgetragen, eine Aussage zu machen, ob New York oder London *hipper* ist. Als erstes müssen wir definieren, wie wir *hip* zu messen gedenken. Wir entscheiden, dass der Grad an *Hippigkeit* gut an der Anzahl der Bart-Träger unter den männlichen flickr-Benützern zu messen ist, und erfinden die folgende, simple Metrik: 

flickr male beard ratio: $\frac{B}{P}$

wobei P die Anzahl der Portraits von Männern auf flickr bezeichnet und
B die Anzahl an Portraits von Männern auf flickr mit Bart.

### Flickr Modul setup
Wir initialisieren den Zugang zum flickr API mit dem folgenden Code:

In [ ]:
from flickrapi import FlickrAPI

FLICKR_PUBLIC = 'XXX'
FLICKR_SECRET = 'XXX'

flickr = FlickrAPI(FLICKR_PUBLIC, FLICKR_SECRET, format='parsed-json')

Nun haben wir ein object *flickr*, mit welchem wir das [flickr API](https://www.flickr.com/services/api/) direkt ansprechen können. [flickr.people.findByUsername](flickr.people.findByUsername) beispielsweise wird wie untenstehend aufgerufen. Ein api_key muss nicht mehr angegeben werden. Will man in einem Argument mehrere Werte übergeben, übergibt man keine Python-Liste sondern man packt alle Werte in einen string und trennt mit Komma.

In [ ]:
flickr.people.findByUsername(username='_rebekka')

Nun geht es los. Fotos finden wir mit [flickr.photos.search](https://www.flickr.com/services/api/flickr.photos.search.html). Wir suchen nach den tags 'portrait,man' und tag_mode='all' (um die beiden angegebenen tags mit AND anstelle von OR zu verknüpfen). Die Region grenzen wir ein, indem wir der Suche die woe_id angeben, welche such auf unsere Zielregion bezieht.

#### Aufgabe 1

In [ ]:
# Finde die woe_id oder place_id von London und schreibe sie als string in die variable woeid_lo
...
woeid_lo = 

#### Vorschlag zur Umsetzung

In [ ]:
# Wir machen die Abfrage...
result = flickr.places.find(query='London', format='parsed-json')
# ...und schauen uns das resultierende json an
result

In [ ]:
# ...und picken den entsprechende Wert raus. 
woe_id_lo = [e['woeid'] for e in result['places']['place'] if e['_content'] == 'London, England, United Kingdom'][0]
woe_id_lo

#### Aufgabe 2

In [ ]:
# Finde die woe_id der Stadt (locality) New York und schreibe sie als string in die variable woeid_ny
...
woe_id_ny = 

#### Vorschlag zur Umsetzung

In [ ]:
# Nochmal dasselbe wie oben
result = flickr.places.find(query='New York', format='parsed-json')
result

In [ ]:
woe_id_ny = [e['woeid'] for e in result['places']['place'] if e['_content'] == 'New York, NY, United States'][0]
woe_id_ny

Mit den nun gefundenen zwei woe_ids können wir unsere Suche starten. Nun schauen wir, ob unsere Abfrage Treffer ergibt und wenn ja, wieviele. Wir suchen mit [flickr.photos.search](https://www.flickr.com/services/api/flickr.photos.search.html) und den tags 'portrait,man' sowie tag_mode='all' in der enstprechenden Region.

#### Aufgabe 3

In [ ]:
# Finde die Anzahl der Treffer zu den angegebenen tags (je für New York und London)
# und schreibe sie in die Variablen num_ny und num_lo
...
num_ny = 
num_lo = 

#### Vorschlag zur Umsetzung

In [ ]:
# Wir suchen mit per_page=1 u die genaue Anzahl an Treffer zu erhalten
num_ny = flickr.photos.search(tags='portrait,man', tag_mode='all', woe_id=woe_id_ny, per_page=1)['photos']['pages']
num_lo = flickr.photos.search(tags='portrait,man', tag_mode='all', woe_id=woe_id_lo, per_page=1)['photos']['pages']
# Falls wir Werte grösser als 4000 bekommen hätten, wäre eine Limitierung auf 4000 notwendig gewesen,
# da das flickr API maximal 4000 Resultate zurückgibt. Zum Beispiel so:
# num_ny = num_ny if num_ny <= 4000 else 4000
# Zur Kontrolle geben wir noch beide Werte aus:
(num_ny, num_lo)

Gut. Nun holen wir pro Region (London und New York) je 100 zufällige Bilder (und sind uns dabei bewusst, dass wir so natürlich überhaupt keine repräsentative Stichprobe erwischen). Leider bietet das flickr API keine Möglichkeit, die Reihenfolge der Resultate zu randomisieren. Deshalb fragen wir pro Ort je 50 API zufällig gewählte page_ids ab mit je nur einem Resultat (Bild) pro Page. Aus diesen Ein-Foto-Pages extrahieren wir anschliessend dann die Links für den Image Download.

Gültige page_ids liegen zwischen 1 und num_ny (bzw num_lo), inklusive.

#### Aufgabe 4

In [ ]:
# Generiere je zufällig 100 page_ids
...
page_ids_ny = 
page_ids_lo = 

#### Vorschlag zur Umsetzung

In [ ]:
import random
# Durch setzen des Status des Pseudo-Zufallszahlengenerators erhalten wir bei jedem Aufruf
# die gleiche Liste an Zufallszahlen. Das hilft normalerweise mit der Reproduzierbarkeit,
# wobei die hier sowieso begrenzt ist, da sich die auf flickr verfügbaren Bilder ja mit
# der Zeit ändern. Wir machen es trotzdem mal.
random.seed(42)

page_ids_ny = random.sample(range(1, num_ny+1), 100)
page_ids_lo = random.sample(range(1, num_lo+1), 100)

# je drei Ausgeben, zum Schauen, ob wir auch was gefunden haben
(page_ids_ny[0:3], page_ids_lo[0:3])

Aus diesen IDs müssen wir nun Download Links generieren. Wir schauen uns als erstes noch einmal eine Response von [flickr.photos.search](https://www.flickr.com/services/api/flickr.photos.search.html) an. Der Download Link befindet sich im Feld *url_m*.

#### Aufgabe 5

In [ ]:
# Verwende flickr.photos.search um die Response für eine einzelne page auszugeben,
# damit wir anschliessend die benötigten Felder heraussuchen können
flickr.photos.search(...)

#### Vorschlag zur Umsetzung

In [ ]:
result = flickr.photos.search(tags='portrait,man', tag_mode='all', per_page=1, page=1, woe_id=woe_id_ny, extras='url_m')
result

Wir sehen, dass die Response aus ein Dictionary mit einem Eintrag mit namen 'photos' besteht. Dieser Eintrag 'photos' wiederum ist eine Liste mit einem einzigen Element, unserem angefragten Photo. Wir benötigen davon nun die 'id', den 'owner' und die direkte URL 'url_m'.

#### Aufgabe 6

In [ ]:
# Stelle die nachstehende Funktion fertig, so das sie verwendet werden kann,
# um eine Liste von Links auf unsere oben generierten Image IDs (page_ids_ny, page_ids_lo) zu generieren.

def create_list_of_images(page_ids, woe_id):
    """ Return a list image urls.
    
    Arguments:
    page_ids -- the list of flickr page ids for which to construct the urls
    woe_id -- the flickr region id to use as a search filter
    
    Returns:
    list of tuples, the first tuple item being the webpage url and the second being the direct url
    """
    list_of_images = []
    for i in page_ids:
        result = flickr.photos.search(...)
        try:
            ...
            photo_id = ...
            user_id = ...
            direct_url = ...
            webpage_url = "https://www.flickr.com/photos/{}/{}".format(user_id, photo_id)
            list_of_images.append((webpage_url, direct_url))
        except IndexError:
            pass # sometimes calls do not return a result
    return list_of_images

#### Vorschlag zur Umsetzung

In [ ]:
def create_list_of_images(page_ids, woe_id):
    """ Return a list image urls.
    
    Arguments:
    page_ids -- the list of flickr page ids for which to construct the urls
    woe_id -- the flickr region id to use as a search filter
    
    Returns:
    list of tuples, the first tuple item being the webpage url and the second being the direct url
    """
    list_of_images = []
    for i in page_ids:
        result = flickr.photos.search(tags='portrait,man', tag_mode='all',
                                      per_page=1, page=i, woe_id=woe_id, extras='url_m')
        try:
            photo = result['photos']['photo'][0]
            photo_id = photo['id']
            user_id = photo['owner']
            direct_url = photo['url_m']
            webpage_url = "https://www.flickr.com/photos/{}/{}".format(user_id, photo_id)
            list_of_images.append((webpage_url, direct_url))
        except IndexError:
            pass # sometimes calls do not return a result
    return list_of_images

Nun generieren wir zwei Listen mit unseren Portraitbildern, einmal für New York und einmal für London.

#### Aufgabe 7

In [ ]:
# Speichere die Liste der Link in den zwei untenstehenden Variablen
ny_images = ...
lo_images = ...

#### Vorschlag zur Umsetzung

In [ ]:
# Die Ausführung dieser Zelle dauert so 1-2 Minuten
ny_images = create_list_of_images(page_ids_ny, woe_id_ny)
lo_images = create_list_of_images(page_ids_lo, woe_id_lo)

# Wiederum je drei Ausgeben, als Kontrolle
(ny_images[0:3], lo_images[0:3])

Schauen wir uns mal ein paar der gefundenen Portraits an.

In [ ]:
from IPython.core.display import HTML
from IPython.display import display
display(HTML(''.join('<a href="{}"><img src="{}" style="width: 180px; \
                      margin: 1px; float: left"/></a>'.format(url[0], url[1]) for url in ny_images[0:25])))

Okay, da ist bei weitem nicht alles brauchbar. Wir haben auch Bilder zurückerhalten, auf denen keine erkennbaren Männer-Gesichter zu sehen sind. Wenn Du auf einige der Bilder klickst, wirst Du sehen, dass tatsächlich bei allen Bildern die Tags 'man' und 'portrait' vergeben wurden. Unsere Abfrage hat also schon korrekt funktioniert, die Daten sind aber (aus unserer Sicht) noch nicht sauber genug.

Wir müssen also noch ein bisschen mehr filtern. Dies machen wir mit der Python-Bibliothek [face_recognition](http://pythonhosted.org/face_recognition/). Face_recognition basiert auf einem (pretrained) neuronalen Netz und bietet ein simples API, um (ohne Cloud-Anbindung) Face Detection und Recognition zu machen.

Dazu holen wir mit der Funktion Befehl [urllib.request.urlopen](https://docs.python.org/3/library/urllib.request.html) für jeden Link eines Bildes ein File-Object mit dem Bild, laden dieses Objekt mit der Funktion [face_recognition.load_image_file()](https://face-recognition.readthedocs.io/en/latest/face_recognition.html#face_recognition.api.load_image_file) in ein array und verwenden [face_recognition.face_locations()](https://face-recognition.readthedocs.io/en/latest/face_recognition.html#face_recognition.api.face_locations) (mit number_of_times_to_upsample=2), um festzustellen, ob auf dem Bild ein erkennbares Gesicht ist.

Beispiele von Face Recognition sind [hier](http://pythonhosted.org/face_recognition/readme.html) zu finden, und das API wird [hier](https://face-recognition.readthedocs.io/en/latest/face_recognition.html) beschrieben.

#### Aufgabe 8

In [ ]:
# Filtere die Listen ny_images und lo_images so, dass face_recognition Gesichter erkennt und
# speichere die Resultate in den folgenden Variablen:

import urllib.request as ur
import face_recognition as fr

...
filtered_ny_images = ...
filtered_lo_images = ...

#### Vorschlag zur Umsetzung

In [ ]:
# Die Ausführung dieser Zelle dauert auch 2-3 Minuten
import urllib.request as ur
import face_recognition as fr

filtered_ny_images = [i[1] for i in ny_images if fr.face_locations(fr.load_image_file(ur.urlopen(i[1])), 1)]
filtered_lo_images = [i[1] for i in lo_images if fr.face_locations(fr.load_image_file(ur.urlopen(i[1])), 1)]

# Das ist etwas unleserlich, hier nochmals mit ausgeschriebenem for loop

#filtered_ny_images = []
#for tup in ny_images:
#    link = tup[1]
#    image = fr.load_image_file(ur.urlopen(link))
#    if fr.face_locations(image, 1):
#        filtered_ny_images.append(link)

Schauen wir uns die gefilterte Liste wiederum an

In [ ]:
display(HTML(''.join('<img src="{}" title="{}" style="width: 180px; \
                      margin: 1px; float: left"/>'.format(url, i) for i, url in enumerate(filtered_ny_images[0:25]))))

Schon viel besser! Die Nicht-Männer filtern wir später.

Dass es Abfrage-bedingt vorkommen kann, dass dieselbe Person mehrmals vorkommt (wenn ein Fotograf von derselben Person sehr viele Portraits gemacht hat), ignorieren wir für diese Übung. Ein Herausfiltern solcher Duplikate wäre mit face_recognition.face_encodings() und face_recognition.compare_faces() möglich, aber aufgrund der vielen notwendigen Vergleiche sehr zeitaufwändig.

Wieviele Bilder sind nun noch übrig?

In [ ]:
(len(filtered_ny_images), len(filtered_lo_images))

Nachdem wir nun unser Set an Daten gesammelt und die Links zu den Bildern in den beiden Variablen `filtered_ny_images` und `filtered_lo_images` gespeichert haben, verwenden wir das [Google Cloud Vision API](https://cloud.google.com/vision/), um für jedes Bild zu entscheiden, ob es einen Mann und einen Bart enthält.

**Achtung**: Für den Workshop stehen **100'000 API Requests**  zur Verfügung (insgesamt, nicht pro Teilnehmer). Bitte gehe sparsam mit den Requests um, damit alle die Übung machen können.

Wir initialisieren den Cloud Vision Client:

In [ ]:
#  API Keys laden
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/data/gcloud.json"

In [ ]:
# Client erstellen
from google.cloud import vision
client = vision.Client()

Nun erstellen wir eine Funktion, die eine Liste von URIs und den initialisierten Client als Parameter nimmt und eine Liste von wahrscheinlichen "Labels" zurückgibt. Labels sind hier diejenigen Dinge, die Google Cloud Vision mit dem analysierten Bild assoziiert.

Mittels `client.image(source_uri='http://...')` wird ein Bild-Objekt erstellt für welches dann mittels `.detect_labels(limit=50)` alle Labels abgefragt werden. Zurück kommt eine Liste von Labels.

Die Elemente dieser Liste sind Label-Objekte, welche das eigentliche Label im Member `.description` haben, und im Member `.score` einen Wert, wie sicher sich Cloud Vision bei diesem Label fühlt (eine Wahrscheinlichkeit zwischen 0 und 1 wobei 1 absolut sicher bedeutet). Wir verwenden diese Score, um nur diejenigen Labels zu berücksichtigen, welche wahrscheinlicher als 0.7 sind.

#### Aufgabe 9

In [ ]:
# Stelle die folgende Funktion wie oben beschrieben fertig
def get_labels(image_uris, client):
    filtered_labels = []
    for u in image_uris:
        image = ... # Bild-Objekt initialisieren
        detected_labels = ... # Labels erkennen
        filtered_labels.append(...) # Liste aller label.description hinzufügen, wenn deren score > 0.7 ist

    assert len(image_uris) == len(filtered_labels) # für jedesd Bild haben wir am Schluss eine Liste von Labels
    return filtered_labels

#### Vorschlag zur Umsetzung

In [ ]:
def get_labels(image_uris, client):
    filtered_labels = []
    for u in image_uris:
        image = client.image(source_uri=u)
        detected_labels = image.detect_labels(limit=50)
        filtered_labels.append([l.description for l in detected_labels if l.score > 0.7])
        
    assert len(image_uris) == len(filtered_labels)
    return filtered_labels

Uns interessiert nun für jedes unserer Bilder, ob Google Cloud Vision ihm die Labels 'man' und 'beard' gibt. Bilder ohne Label 'man' ignorieren wir, denn das sind (im Rahmen unseres simplen Beispiels) solche, auf denen kein Mann drauf ist.

Aus den Bildern mit den labels 'man' und 'beard' errechnen wir nun für New York und London je die anfangs definierte *flickr male beard ratio*.

#### Aufgabe 10

In [ ]:
# Berechne je Stadt die flickr male beard ratio mit den Listen filtered_ny_images und filtered_lo_images
# Um unnötige Wartezeiten und unnötigen Verbrauch von Requests zu vermeiden,
# führe die Request an Google nur einmal durch

# Die Ausführung dieser Zelle dauert auch 3-4 Minuten
ny_labels = get_labels(filtered_ny_images, client)
lo_labels = get_labels(filtered_lo_images, client)

In [ ]:
# Arbeite in dieser Zelle weiter, damit Du die Requests nicht mehrfach ausführen musst
...

#### Vorschlag zur Umsetzung

In [ ]:
ny_total_males = sum([1 for label_list in ny_labels if 'man' in label_list])
ny_bearded_males = sum([1 for label_list in ny_labels if 'man' in label_list and 'beard' in label_list])
lo_total_males = sum([1 for label_list in lo_labels if 'man' in label_list])
lo_bearded_males = sum([1 for label_list in lo_labels if 'man' in label_list and 'beard' in label_list])

In [ ]:
flickr_male_beard_ratio_ny = ny_bearded_males/ny_total_males
flickr_male_beard_ratio_lo = lo_bearded_males/lo_total_males
(flickr_male_beard_ratio_ny, flickr_male_beard_ratio_lo)

Und nun wissen wir, welche der beiden Städte hipper ist und hätten auch diese unglaublich wichtige Frage geklärt!